# Ollama + OpenAI + Python

## 1. 指定模型名稱

如果你使用的模型不是 "phi3:mini"，請在下面的單元格中更改該值。
這個變數將在整個筆記本的程式碼中使用。


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. 設置 Open AI 客戶端

通常 OpenAI 客戶端會與 OpenAI.com 或 Azure OpenAI 一起使用，以便與大型語言模型互動。
然而，它也可以與 Ollama 一起使用，因為 Ollama 提供了一個與 OpenAI 相容的端點，位於 "http://localhost:11434/v1"。


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. 生成聊天回覆

現在我們可以使用 OpenAI SDK 為對話生成回覆。這個請求應該生成一首關於貓的俳句：


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. 提示工程

發送給語言模型的第一條訊息被稱為「系統訊息」或「系統提示」，它設定了模型的整體指示。
你可以提供自訂的系統提示，指導語言模型以不同的方式生成輸出。
修改以下的 `SYSTEM_MESSAGE`，讓模型以你最喜愛的電影/電視角色的方式回答，或者從 [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts) 中獲取其他系統提示的靈感。

當你完成自訂系統訊息後，請在 `USER_MESSAGE` 中提供第一個使用者問題。


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. 少量示例

另一種引導語言模型的方法是提供「少量示例」，即一系列示範問題和答案，展示模型應如何回應。

以下示例嘗試讓語言模型扮演教學助理的角色，透過提供一些教學助理可能會回答的問題和答案，然後以學生可能提出的問題來提示模型。

先試試看，然後修改 `SYSTEM_MESSAGE`、`EXAMPLES` 和 `USER_MESSAGE` 以適應新的場景。


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. 檢索增強生成

RAG（檢索增強生成）是一種技術，用於讓語言模型在特定領域內準確回答問題。其方法是先從知識來源中檢索相關資訊，然後根據該資訊生成回應。

我們提供了一個包含混合動力車數據的本地 CSV 文件。以下程式碼會讀取該 CSV 文件，搜尋與使用者問題匹配的內容，然後根據找到的資訊生成回應。請注意，這個過程會比之前的例子耗時更長，因為它向模型傳送了更多數據。如果你發現答案仍然未能基於數據，則可以嘗試系統工程或使用其他模型。一般來說，RAG 在使用更大的模型或經過微調的 SLM 版本時效果更佳。


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**免責聲明**：  
本文件已使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。原始語言的文件應被視為權威來源。對於重要資訊，建議使用專業的人類翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋概不負責。
